# Presentation Structure

## Data Analytics Lifecycle
### 1. Define
- Pre-done

### 2. Gather Data
- Pre-done

### 3. Cleaning
1. Standardize Columns to aggregate (sort & lowercase & concatenate files)
2. ST Column -> rename to state column
3. GENDER Column -> rename to gender column
4. Gender Column -> Standardization column to only have "Female" and "Male"
5. State column -> Add extra column to show "West", "East", "North", "South" regions
6. Transform columns into their intended data type (e.g. remove "%" and extract 0/0/00)
7. Drop duplicated rows
8. Drop data values that show outliers with the help of IQR method

### 4. Exploring
- Keep business objectives in mind:
    1. Retain customers
    2. Analyze relevant customer data
    3. Develop focused customer retention programs
- Customer Lifetime Value Analysis: Column Monthly_premium_auot <-> Column CLF show highest correlation
- Region Analysis
- Education Analysis

### 5. Processing
- upcoming in future weeks

### 6. Apply Model
- upcoming in future weeks

### 7. Validate
- upcoming in future weeks

### 8. Present
- upcoming in future weeks

## Weeky 2 - Monday: Activity 1 

### Task 1: Aggregate the data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them.

In [ ]:
#Importing neccessary Python libraries (e.g. Pandas, Numpy, Matplotlib, Seaborn)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#importing file1 and previewing setup
#Anomalies: column "ST" and "GENDER"
file1 = pd.read_csv("file1.csv")
file1.head()

In [ ]:
#importing file1 and previewing setup
#Anomalies: column "ST" and "GENDER"
file2 = pd.read_csv("file2.csv")
file2.head()

In [ ]:
#importing file1 and previewing setup
#Anomalies: No anomalies found in column's names
file3 = pd.read_csv("file3.csv")
file3.head()

### Task 2: Standardizing header names

In [ ]:
#Functions to standardize file1, file2, file3 headers names and hence fix found anomalies 
#Function to lower case the column names -> allows for sorting them and hence aggregating them more easily
def lower_case_column_names(dataframe):
    dataframe.columns=[i.lower() for i in dataframe.columns]
    return dataframe

#Function to clean "st" column in file1 and file2
def clean_state_column(dataframe):
    dataframe.rename(columns={'st':'state'}, inplace=True)
    return dataframe

In [ ]:
#Applying above created functions to allow for concatenating file1, file2, file3
lower_case_column_names(file1)
lower_case_column_names(file2)
lower_case_column_names(file3)
clean_state_column(file1)
clean_state_column(file2)

In [ ]:
#Sorting dataframes to hence allow for concatenating
file1 = file1.sort_index(axis=1)
file2 = file2.sort_index(axis=1)
file3 = file3.sort_index(axis=1)

In [ ]:
#Double check if all columns are equal. If yes -> ready for concatenating!
file1.columns == file2.columns
file2.columns == file3.columns
file1.columns == file3.columns

In [ ]:
#Aggregate the data into one new dataframe, called "df"
df = pd.concat([file1,file2,file3], axis=0)

In [ ]:
#Gathering some basic information about concatenated dataset
df.shape

### Task 3: Removing duplicates

In [ ]:
#Removing duplicates
df = df.drop_duplicates()

### Task 4: Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data

In [ ]:
#Deleting and rearranging the column "customer"
df = df.drop("customer", axis = 1)

In [ ]:
#Reset the index after recently deleted "customer" column
df.reset_index(drop=True, inplace=True)

### Task 5: Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.

In [ ]:
#Gathering information about each of the df's columns and their data type
#Anomalies: 
    #1) "customer lifetime value"-column is of type string/object whereas the preview states soly integers => requires analysis
    #2) "number of open complaints"-column is of type string/object whereas the preview states integers => requires analysis
df.info()

### Task 5.1: Customer Lifetime Value Column Analysis

In [ ]:
#Apply lamda function to delete "&" signs of "customer lifetime value" column
df["customer lifetime value"] = df["customer lifetime value"].apply(lambda x: str(x).replace('%', ''))
df["customer lifetime value"] =  pd.to_numeric(df["customer lifetime value"], errors='coerce')

#Double check if datatype of column has changed from object to numeric (int or float). Otherwise, further analysis required
df.info()

In [ ]:
#Double check, how many NaN values are still present in the "customer lifetime value"-column
#Food for thought: might delete NaN in later stage of EDA if necessary
df[df["customer lifetime value"].isnull() == True]

### Task 5.2: Number of Open Complaints Analysis / clean the number of open complaints and extract the middle number which is changing between records.

In [ ]:
#Translate all values (str/int/float etc) of "Number of Open Complaints"-column into numeric data column, and ignore any upcoming error
df["number of open complaints"] =  pd.to_numeric(df["number of open complaints"], errors='ignore')

#Double check, which values have not been transformed into numeric values and count these
df[df["number of open complaints"].str.isnumeric() == False]
df[df["number of open complaints"].str.isnumeric() == False]["number of open complaints"].value_counts()

#Next steps: find method with which you can extract middle part of x/x/xx format

In [ ]:
#Identified concept on how to pull second value between / and / and transform value into int (in principle)
df["number of open complaints"][0].split("/")

In [ ]:
#Apply above identified concept on all entries in the "number of open complaints"-column, using the apply function
df["number of open complaints"] = df["number of open complaints"].apply(lambda x: int(x.split('/')[1]) if type(x) == str else x)

In [ ]:
#Double check if now all values of "number of open complaints"-column are of type int
df["number of open complaints"].value_counts()

In [ ]:
#Double check if now all values of "number of open complaints"-column are of type int
df.info()

### Task 6: Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those column

In [ ]:
#First identifying the unique options in the "gender"-column
df["gender"].value_counts()

In [ ]:
#First identifying the unique options in the "state"-column
df["state"].value_counts()

In [ ]:
#Function, that standardizes the column "gender" based on the in the previous step identified unique categories
def standardize_gender(x):
    if x in ["F", "female", "Femal"]:
        return "Female"
    elif x in ["M", "Male"]:
        return "Male"
    else:
        return x

In [ ]:
#Function, that standardizes the column "state" based on the in the previous step identified unique categories
def standardize_state(x):
    if x in ["California", "Cali"]:
        return "California"
    elif x in ["Arizona", "AZ"]:
        return "Arizona"
    elif x in ["Washington", "WA"]:
        return "Washington"
    else:
        return x

In [ ]:
#Applying the previously built function
df["gender"] = df["gender"].apply(standardize_gender)
df["state"] = df["state"].apply(standardize_state)

In [ ]:
#Double checking if applied function works as intended
df["state"].value_counts()
df["state"].value_counts()

In [ ]:
#Print out the final dataset to double check for any other anomanlies
df

## Weeky 2 - Tuesday: Activity 2

In [ ]:
##Replacing null values – Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values) Hint: numpy.nan is considered of float64 data type.
##Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central
##(Optional) In the column Vehicle Class, merge the two categories Luxury SUV and Luxury Car into one category named Luxury Vehicle
##(Optional) Removing outliers using 1.5*IQR technique for all numerical columns.
##(Optional) Standardizing the data – Use string functions to standardize the text data (lower case)

### Task 1: Replacing null values – Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values) Hint: numpy.nan is considered of float64 data type.

In [ ]:
#Function to replace NaN values in numerical functions
def replace_with_mean(dataframe, clmn):
    
    if dataframe[clmn].dtype != object:
        mean_value = dataframe[clmn].mean()
        dataframe[clmn].fillna(value = mean_value, inplace = True)
    else:
        pass

In [ ]:
#Apply previously built function to all numeric columns
#Double check if in all numeric columns the NaN values have been removed (via checking if the len(isna()) is 0.
for column in df.columns:
    replace_with_mean(df,column)
    if len(df[column].isna().value_counts()) <= 1:
        print(f"The {column} has been successfully transformed")

### Task 2: Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central

In [ ]:
#Build translator dictionary and consequently replace those in the dataframe
translator = {"California": "West Region", 
                "Oregon": "North West Region", 
                "Washington": "East Region", 
                "Nevada": "Central",
                "Arizona": "Central"}

df["state"] = df["state"].replace(translator)

### Task 3: (Optional) In the column Vehicle Class, merge the two categories Luxury SUV and Luxury Car into one category named Luxury Vehicle

In [ ]:
#Build translator dictionary and consequently replace those in the dataframe
translator_dict = {"Luxury SUV": "Luxury Vehicle",
            "Luxury Car": "Luxury Vehicle"}

df["vehicle class"] = df["vehicle class"].replace(translator_dict)

### Task 4: (Optional) Removing outliers using 1.5*IQR technique for all numerical columns.

In [ ]:
#Proof shape of df before outlier removal
df.shape

In [ ]:
#Function that defines upper and lower limits based on IQR method and consequently replaces lower / larger values with NaN
#Function does NOT drop the whole row when there is a NaN found in one column, but instead only replaces the value
def remove_outlier(column_name, df):
    percentile25 = df[column_name].quantile(0.25)
    percentile75 = df[column_name].quantile(0.75)
    iqr = percentile75 - percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    
    df_droppers_upper = df[df[column_name] > upper_limit].index
    df_droppers_lower = df[df[column_name] < lower_limit].index
    total_to_drop = df_droppers_upper.union(df_droppers_lower)
    
    df = df.drop(total_to_drop, inplace=True)
    
    return df

In [ ]:
#Apply function to all columns in df that are of numerical datatype
for column in df.columns:
    if df[column].dtype != object:
        remove_outlier(column, df)

In [ ]:
#Double check if outlier removal has been successfull: shape previous (9135 rows) vs shape after (5221 rows)
df.shape

In [ ]:
#Reset index
df.reset_index()

### Task 5: (Optional) Standardizing the data – Use string functions to standardize the text data (lower case)

In [ ]:
#Function that lower cases the entries in each column that is of type object
def standardize(df, clmn):
    if df[clmn].dtype == object:
        df[clmn] = df[clmn].str.lower()
    else:
        pass
    return df

In [ ]:
#Applying the function to each column in the dataframe
for column in df.columns:
    standardize(df,column)

## Weeky 2 - Wednesday: Activity 3

### Task 1: Get the numeric data into dataframe called numerical and categorical columns in a dataframe called categoricals. (You can use np.number and np.object to select the numerical data types and categorical data types respectively)

In [ ]:
#Previewing original data
new_df = pd.read_csv("/Users/lukasbauerschmidt/Desktop/Ironhack/2. Class Materials/IH_DA_FT_JAN_2023/Class_Materials/Case_Studies/Customer_Analysis_Case_Study/Data/Data_Marketing_Customer_Analysis_Round3.csv")
new_df.head()

In [ ]:
#Splitting data into numerical only features, allowing for targeted processing and EDA
num_df = new_df.select_dtypes(include=['int', 'float'])
num_df.head()

In [ ]:
#Splitting data into caterogical only features, allowing for targeted processing and EDA
cat_df = new_df.select_dtypes(exclude=['int', 'float'])
cat_df.head()

### Task 2: Now we will try to check the normality of the numerical variables visually

### Task 2.1: Use seaborn library to construct distribution plots for the numerical variables

In [ ]:
#Plot histogram with seaborn for each column in the numerical dataframe with for loop
fig=plt.figure(figsize=(20,15))
for i, column in enumerate(num_df.columns, 1):
    plt.subplot(3,3,i)
    sns.histplot(num_df[column])

### Task 2.2.: Use Matplotlib to construct histograms.

In [ ]:
#Plot histogram with matplotlib for each column in the numerical dataframe with for loop
fig=plt.figure(figsize=(20,20))
for i, column in enumerate(num_df.columns, 1):
    plt.subplot(3,3,i)
    plt.hist(num_df[column], edgecolor = "black")
    plt.title(column)
    plt.xlabel(column)
    plt.ylabel("count")

### Task 2.3: Do the distributions for different numerical variables look like a normal distribution?

In [ ]:
#Customer lifetime value & total amount claimed seem to be variables which show right skewed data distribution
#Months since policy inception and months since last claim seem to be also close to normal distribution

### Task 3: For the numerical variables, check for correlation between the input features. Note: this does not include the target feature. Plot the Correlation Heatmap.

In [ ]:
#Drop the target variable from the numerical dataframe
num_df_wo_target = num_df.drop("total_claim_amount", axis=1)

#Built correlation matrix of numerical dataframe (w/o target variable)
corr = num_df_wo_target.corr()

#Plot and layout heatmap using seaborn
plt.figure(figsize=(10, 4))
heatmap = sns.heatmap(corr, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

### Task 4: Additional Analysis

In [ ]:
sns.pairplot(new_df, hue="region", height=3)

## Weeky 2 - Thursday: Activity 4¶

### Task 1: Show a plot of the total number of responses.

### Task 2: Show a plot of the response by the sales channel.

### Task 3: Show a plot of the response by the total claim amount.

### Task 4: Show a plot of the response by income.

In [ ]:
#Creating extra column in dataframe, that translates responses from boolean to numerical values
translator = {"no": 0, "yes": 1}
new_df["response_num"] = new_df["response"].replace(translator)

In [ ]:
#Create 2x2 plot figure
fig=plt.figure(figsize=(20,10))
plt.suptitle('Activity 4 Plot Overview', fontsize=18)

#Plot 1 (0,0) for Task 1
plt.subplot(3,3,1)
y=cat_df["response"].value_counts()
x=y.index
plt.bar(x,y, color = ["red", "green"])
plt.title('Task 1: Total Number of Responses', fontsize=12, fontweight = "bold")
plt.xlabel("count")
plt.ylabel("response")

#Plot 2 (0,1) for Task 2
plt.subplot(3,3,2)
plt.title('Task 2: Response by Sales Channel', fontsize=12, fontweight = "bold")
sale_by_channel = new_df.groupby(["sales_channel", "response"])["response_num"].count().reset_index()
sns.barplot(x = 'sales_channel',y = 'response_num', hue = 'response', data = sale_by_channel, palette=["red", "green"])

#Plot 3 (1,0) for Task 3
plt.subplot(3,3,3)
plt.title('Task 3: Response by Total Claim Amount', fontsize=12, fontweight = "bold")
sns.histplot(x="total_claim_amount", hue="response", data = new_df, palette=["red", "green"])

#Plot 4 (1,1) for Task 4
plt.subplot(3,3,4)
plt.title('Task 4: Response by Income', fontsize=12, fontweight = "bold")
sns.histplot(x="income", hue="response", data = new_df, palette=["red", "green"])

plt.subplot(3,3,5)
plt.title('Task 4: Response by Area', fontsize=12, fontweight = "bold")
sns.histplot(x="region", hue="response", data = new_df, palette=["red", "green"])


plt.subplot(3,3,6)
plt.title('Task 4: Response by Education', fontsize=12, fontweight = "bold")
sns.histplot(x="education", hue="response", data = new_df, palette=["red", "green"])

plt.subplot(3,3,6)
plt.title('Task 4: Total Claim Amount by Education', fontsize=12, fontweight = "bold")
sns.histplot(x="education", hue="total_claim_amount", data = new_df, palette=["red", "green"])


#Adjusting layout to avoid overlap
plt.tight_layout()

### Task 5: (Optional) plot any interesting findings/insights(minimum three) that describe some interesting facts about your data set and its input variables as well as relationships with the target feature.

In [ ]:
plt.subplots(figsize = (30,20))
for i, column in enumerate(cat_df.columns, 1):
    plt.subplot(5,5,i)
    plt.title(f"{column}")
    sns.countplot(x=cat_df[column], data=cat_df)
    

plt.tight_layout()

In [ ]:
policy_response_survival = new_df.groupby(['policy_type','response'])['response_num'].count().reset_index()
sns.barplot(x = policy_response_survival['policy_type'],y = policy_response_survival['response_num'], hue = policy_response_survival['response'])

In [ ]:
policy_response_survival = new_df.groupby(['vehicle_size','response'])['response_num'].count().reset_index()
sns.barplot(x = policy_response_survival['vehicle_size'],y = policy_response_survival['response_num'], hue = policy_response_survival['response'])

In [ ]:
plt.subplots(figsize = (30,20))
for i, column in enumerate(num_df.columns, 1):
    plt.subplot(2,4,i)
    plt.title(f"{column}")
    #sns.countplot(x=cat_df[column], data=cat_df)
    sns.scatterplot(x=new_df["monthly_premium_auto"], y=new_df["customer_lifetime_value"], hue=new_df[column])

## Week 4 - Monday: Activity 5

### Task 1: check if there are highly correlated features and drop them if there are any.

In [3]:
#Importing neccessary Python libraries (e.g. Pandas, Numpy, Matplotlib, Seaborn)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#Previewing original data
df_5 = pd.read_csv("/Users/lukasbauerschmidt/Desktop/Ironhack/2. Class Materials/IH_DA_FT_JAN_2023/Class_Materials/Case_Studies/Customer_Analysis_Case_Study/Data/Data_Marketing_Customer_Analysis_Round3.csv")
df_5.head()

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize


In [5]:
df_5_num = df_5.select_dtypes(include=['int', 'float'])
df_5_cat = df_5.select_dtypes(exclude=['int', 'float'])

In [ ]:
#Built correlation matrix of numerical dataframe (w/o target variable)
corr = df_5_num.corr()
corr

#monthly premium auto feature seems to be the only feature which shows "higher" correlation. Hence we drop this feature

In [ ]:
df_5_num.drop("monthly_premium_auto", axis=1)

### Task 2: One Hot/Label Encoding of the categorical variables in the categoricals data frame that you created in Activity 3.

In [ ]:
#Cleaning dataset and distinguishing between which categorical columns are nominal and ordinal
df_5_cat.head()

#drop the effective_to_date column since it is indeed numerical
df_5_cat.drop("effective_to_date", axis=1)
nominal = ["region", "response", "employment_status", "gender", "location_code", "marital_status", "policy_type", "policy", "renew_offer_type", "sales_channel", "vehicle_class"]
ordinals = ["coverage", "education", "month", "vehicle_size"]

In [ ]:
#Encoding categorical - nominal columns
import sklearn
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False, drop = "first")
encoder_vars_array = one_hot_encoder.fit_transform(df_5_cat[nominal])
encoder_feature_names = one_hot_encoder.get_feature_names_out(nominal)
encoder_vars_df = pd.DataFrame(encoder_vars_array, columns = encoder_feature_names)
encoder_vars_df

In [ ]:
df_5[ordinals].value_counts()

In [ ]:
df_5["vehicle_size"].value_counts()

In [ ]:
df_5_cat[ordinals]

In [ ]:
#Encoding categorical - ordinal columns
from sklearn.preprocessing import OrdinalEncoder

maplist = {'coverage': {'basic': 0, 'extended': 1, 'premium': 2},
           'education': {'high school or below': 0, 'bachelor': 1, 'college': 2, 'master': 3, 'doctor': 4},
           'month': {'jan': 0, 'feb': 1},
           'vehicle_size': {'small': 0, 'medsize': 1, 'large': 2,}}
df_5_cat_ordinary_encoded = df_5_cat[ordinals].apply(lambda x: x.map(maplist[x.name]))
df_5_cat_ordinary_encoded

In [ ]:
df_5_cat_ordinary_encoded.head()

In [ ]:
encoder_vars_df.head()

In [ ]:
#Concatenating two previously encoded/generated dataframes
df_5_cat_encoded_total = pd.concat([df_5_cat_ordinary_encoded.reset_index(drop=True), encoder_vars_df.reset_index(drop=True)], axis = 1)
df_5_cat_encoded_total

### Optional Task

### Task 1: Variable A

In [ ]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler

In [ ]:
#Checking which column shows widest range
#Income variable has largest range
(df_5_num.max() - df_5_num.min()).idxmax()

In [ ]:
varA = df_5_num["income"]

In [ ]:
#Checking which column shows largst skewness
#customer_lifetime_value variable has largest skew
(df_5_num.max() - df_5_num.mean()).idxmax()

In [ ]:
varB = df_5_num["customer_lifetime_value"]

In [ ]:
#before transformation: varA distribution
sns.histplot(varA)

In [ ]:
varA_reshaped = varA.to_numpy().reshape(-1,1)
minmaxscaler = MinMaxScaler()
varA_transformed = minmaxscaler.fit_transform(varA_transformed)

In [ ]:
##after transformation: varA distribution
sns.histplot(varA_transformed)

### Task 2: Variable B

In [ ]:
#before transformation: varB distribution
sns.histplot(varB)

In [ ]:
varB_rehaped = varB.to_numpy().reshape(-1,1)
varB_rehaped

In [ ]:
#powertransformer = PowerTransformer(method='yeo-johnson')
powertransformer = PowerTransformer(method='box-cox')
varB_power_transformed = powertransformer.fit_transform(varB_rehaped)

In [ ]:
#after power transformation: varB distribution
sns.histplot(varB_power_transformed)

In [ ]:
print(varB_power_transformed.mean(), varB_power_transformed.std())

In [ ]:
#after standard scaler: varB distribution
standardscaler = StandardScaler()
varB_standard_scaled = standardscaler.fit_transform(varB_rehaped)

In [ ]:
sns.histplot(varB_standard_scaled)

In [ ]:
print(varB_standard_scaled.mean(), varB_standard_scaled.std())

## Weeky 4 - Tuesday: Activity 6¶

In [6]:
#Import of necessary libraries
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler

In [7]:
#Only assume numerical data to be revelant (for now leave out the categorical)
df_5_num

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,92260,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...
10684,15563,61541,253,12,40,0,7,1214
10685,5259,61146,65,7,68,0,6,273
10686,23893,39837,201,11,63,0,2,381
10687,11971,64195,158,0,27,4,6,618


In [8]:
#Splitting data into x and y 
x = df_5_num.drop("total_claim_amount", axis=1)
y = df_5_num["total_claim_amount"]

In [9]:
#Splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state = 42)

In [27]:
#Standardizing the dataset
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [43]:
#Apply linear regression
#Scikit Learn Method
model = LinearRegression()
model.fit(x_train_scaled, y_train)
model.predict(x_test_scaled)

array([230.08062389, 559.06281849, 340.97224399, ..., 490.61253546,
       463.72545559, 590.35490144])

In [35]:
#Scikitlearn Model Interpretation: Betas, Intercepts
print(model.coef_, model.intercept_)

[-7.44374050e+00 -3.35677864e+01  1.88742534e+02  1.27912773e-01
 -3.00331832e+00  2.86638652e-01  2.40707525e+00] 432.7827154718746


In [54]:
#Scikitlearn Model Interpretation: R2
model.score(x_test_scaled, y_test)

0.40127309153524027

In [36]:
#Import of necessary libraries
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [55]:
#Apply linear regression
#OLS Method
X_train_const = sm.add_constant(x_train_scaled)
X_test_const = sm.add_constant(x_test_scaled)

model_OLS = sm.OLS(y_train, X_train_const).fit()
model_OLS.predict(X_test_const)

array([230.08062389, 559.06281849, 340.97224399, ..., 490.61253546,
       463.72545559, 590.35490144])

In [56]:
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:     total_claim_amount   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     853.4
Date:                Tue, 31 Jan 2023   Prob (F-statistic):               0.00
Time:                        18:10:11   Log-Likelihood:                -58478.
No. Observations:                8551   AIC:                         1.170e+05
Df Residuals:                    8543   BIC:                         1.170e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        432.7827      2.443    177.124      0.0